# Assignment 4

### Instructions

* Write your code in the cells provided.  Where appropirate, enter markdown to answer questions.

* Submit this notebook to owl.

---


In [1]:
# It's dangerous to go alone.  Take these!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer
pd.set_option('display.max_columns', 500)

%matplotlib inline

### You're A Data Scientist!
You are working as a Junior Data Scientist for a professional football (er, Soccer) club.  The owner of the team is very interested in seeing how the use of data can help improve the team's peformance, and perhaps win them a championship!

The draft is coming up soon (thats when you get to pick new players for your team), and the owner has asked you to create a model to help score potential draftees.  The model should look at attributes about the player and predict what their "rating" will be once they start playing professionally.

The football club's data team has provided you with data for 17,993 footballers from the league.  Your job: work with the Senior Data Scientist to build a model or models, perform model selection, and make predictions on players you have not yet seen.

### The Dataset

The data is stored in a csv file called `footballer_data.csv`.  The data contain 52 columns, including some information about the player, their skills, and their overall measure as an effective footballer.

Most features relate to the player's abilities in football related skills, such as passing, shooting, dribbling, etc.  Some features are rated on a 1-5 scale (5 being the best), others are rated on 0-100 (100 being the best), and others still are categorical (e.g. work rate is coded as low, medium, or high).

The target variable (or $y$ variable, if you will) is `overall`.  This is an overall measure of the footballer's skill and is rated from 0 to 100.  The most amazingly skilled footballer would be rated 100, where as I would struggle to score more than a 20. The model(s) you build should use the other features to predict `overall`.



### Part A

Read in the data and take a look at the dataframe.  There should be 52 columns. The outcome of interest is called `overall` which gives an overall measure of player performance. Not all of the other columns are particularly useful for modelling though (for instance, `ID` is just a unique identifier for the player.  This is essentially an arbitrary number and has no bearing on the player's rating).

The Senior Data Scientist thinks the following columns should be removed:

* ID
* club
* club_logo
* birth_date
* flag
* nationality
* photo
* potential

The Senior Data Scientist would also like the following columns converted into dummy variables:

* work_rate_att
* work_rate_def
* preferred_foot

Clean the data according to the Senior Data Scientist's instructions.

In [ ]:
footballer_out = pd.read_csv('footballer_data.csv')
footballer_out_drop = footballer_out.drop(['ID', 'club', 'club_logo', 'birth_date', 'flag', 'nationality', 'photo', 'potential'], axis = 'columns')
model_drop_dummy = pd.get_dummies(footballer_out_drop, drop_first=True)
display(model_drop_dummy)

### Part B

The data should all be numerical now. Before we begin modelling, it is important to obtain a baseline for the accuracy of our predictive models. Compute the absolute errors resulting if we use the median of the `overall` variable to make predictions. This will serve as our baseline performance. Plot the distribution of the errors and print their mean and standard deviation.

In [ ]:
overall_median  = model_drop_dummy['overall'].median()
print(overall_median)
column_needed_series = pd.Series(model_drop_dummy['overall'])
column_needed = (column_needed_series).to_numpy()
abs_error = np.absolute(column_needed  - overall_median )
abs_error_mean = np.mean(abs_error)
abs_error_std = np.std(abs_error)
print(f"mean: {abs_error_mean}")
print(f"standard deviation: {abs_error_std}")
sns.distplot(abs_error)

### Part C
To prepare the data for modelling, the Senior Data Scientist recomends you use `sklearn.model_selection.train_test_split` to seperate the data into a training set and a test set.

The Senior Data Scientist would like to estimate the performance of the final selected model to within +/- 0.25 units using mean absolute error as the loss function of choice.  Decide on an appropriate size for the test set, then use `train_test_split` to split the features and target variables into appropriate sets.

In [ ]:
std_final = abs_error_std
sample_size = (2*std_final/0.25)**2
print(f"sample-size: {sample_size}")
y = model_drop_dummy.overall
X = model_drop_dummy.drop('overall', axis = 'columns')
# Split into train&validation, test
# Random state assures that folds are consistent across models
Xtrainval, Xtest, ytrainval, ytest = train_test_split(X,y, test_size = 1162, random_state = 0)
print(Xtrainval.shape,Xtest.shape, ytrainval.shape, ytest.shape)

### Part D

The Senior Data Scientist wants you to fit a linear regression to the data as a first model.  Use sklearn to build a model pipeline which fits a linear regression to the data. (This will be a very simple, one-step pipeline but we will expand it later.) You can read up on sklearn pipelines [here](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Note that the sklearn linear regression adds its own intercept so you don't need to create a column of 1s.

In [ ]:
steps = [('LR', LinearRegression())]
pipeline = Pipeline(steps)
pipline_model  = pipeline.fit(Xtrainval, ytrainval)
print(pipline_model)

## Part E

The senior data scientist wants a report of this model's cross validation score.  Use 5 fold cross validation to estimate the out of sample performance for this model.  You may find sklearn's `cross_val_score` useful.

In [ ]:
cv_scores = cross_val_score(pipline_model, Xtest, ytest, 
                            cv=5, scoring=make_scorer(mean_absolute_error))
print(cv_scores)

### Part F

That's impressive!  Your model seems to be very accurate, but now the Senior Data Scientist wants to try and make it more accurate.  Scouts have shared with the Senior Data Scientist that players hit their prime in their late 20s, and as they age they become worse overall.

The Senior Data Scientist wants to add a quadratic term for age to the model.  Repeat the steps above (creating a pipeline, validating the model, etc) for a model which includes a quadratic term for age.

In [ ]:
Xtrainval2 = Xtrainval
Xtrainval2 = Xtrainval2.assign(age2 = Xtrainval.age**2)
steps_new = [('LR', LinearRegression())]
pipeline_new = Pipeline(steps_new)
pipline_model_new  = pipeline_new.fit(Xtrainval2, ytrainval)
print(pipline_model_new)
#linmodel = LinearRegression().fit(Xtrainval,ytrainval)
#print(linmodel)
cv_scores_new = cross_val_score(pipline_model_new, Xtest, ytest, 
                            cv=5, scoring=make_scorer(mean_absolute_error))
print(cv_scores_new)

### Part G


The Senior Data Scientist isn't too happy that the quadratic term has not improved the fit of the model much and now wants to include quadratic and interaction term for every feature (That's a total of 1080 features!!!!)

Add sklearn's `PolynomialFeatures` to your pipeline from part C.  Report the cross validation score.

In [ ]:
pipeline_new_features = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('linear_regression', LinearRegression())
])
Xtrianval_new=Xtrainval
pipline_model_new_features = pipeline_new_features.fit(Xtrianval_new, ytrainval)

cv_scores_new_features = cross_val_score(pipline_model_new_features,Xtrianval_new ,  ytrainval, 
                            cv=5, scoring=make_scorer(mean_absolute_error))
print(f"CrossValidationScore: {cv_scores_new_features}")
print(f"CrossValidationError: {cv_scores_new_features.mean()}")

In [ ]:
### Part H

The Senior Data Scientist is really happy with the results of adding every interaction into the model and wants to explore third order interactions (that is adding cubic terms to the model).

This is not a good idea!  Talk them down from the ledge.  Write them an email in the cell below explaining what could happen if you add too may interactions.

---

Hey Boss,

I got your email about adding cubic terms to the model. I think it might increase the performance but increases complexcity and might result in overfitting as well. When we use cubic terms the total number of columns is 46+C(46,3) = 91,126

Sincerly,

Junior Data Scientist



### Part I

You've successfully talked the Senior Data Scientist out of adding cubic terms to the model. Good job!

Based on the cross validation scores, which model would you choose?  Estimate the performance of your chosen model on the test data you held out, and do the following:

- Compute a point estimate for the generalization error.
- Compute a confidence interval for the generalization error.  
- Plot the distribution of the absolute errors.

Is the test error close to the cross validation error of the model you chose? Why do you think this is the case?


In [ ]:
pipline_model_new_features  = pipeline_new_features.fit(Xtrianval_new, ytrainval)
AbsoluteErrors= np.abs(pipline_model_new_features.predict(Xtest) - ytest)
point_Estimation_GE = np.mean(AbsoluteErrors)
standard_deviation = AbsoluteErrors.std()
Confidence_Interval = [point_Estimation_GE-(2*standard_deviation/np.sqrt(1162)), point_Estimation_GE+(2*standard_deviation/np.sqrt(1162))]
print(f"ConfidenceInterval: {Confidence_Interval}")
print(f"PointEstimateGeneralizationError: {point_Estimation_GE}")
print(f"GeneralizationError: {AbsoluteErrors}")
sns.distplot(AbsoluteErrors)

In [ ]:
a)I chose the second one. In fact, after adding quadratic feature, the model performance improves.
b) Yes the test error is close to cross validation error. Maybe because we do cross validation error at once and we have used enough training data. I beleive the median we used for prediction plays a role as well.